<a href="https://colab.research.google.com/github/Vl-Leschinskii/jokes_topics/blob/main/Anekdot_for_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Нужные установки

In [ ]:
!pip3 install pyLDAvis

In [ ]:
!pip install transformers sentencepiece

In [ ]:
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer

parser = MorphAnalyzer()

In [ ]:
import re

In [ ]:
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами
import matplotlib.pyplot as plt  # Пакет для строительства графиков
import seaborn as sns            # Еще один пакет для графиков
import pickle                    # Пакет для подгрузки данных специфического для питона формата

#from dateutil.relativedelta import relativedelta # еще один пакет для работы с временем
from wordcloud import WordCloud  # Пакет для построения облаков слов

plt.style.use('ggplot')          # задаём красивый стиль для графиков matplotlib-а
%matplotlib inline

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import os
import requests
from pathlib import Path
import nltk
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from collections import Counter
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize 
from nltk import download as nltk_download 

### Функции

In [ ]:
def embed_bert_cls(text, model, tokenizer):
#    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    t = tokenizer(text, padding=True, truncation=False, return_tensors='pt')

    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [ ]:
def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)

    if n1 < 1e-6 or n2 < 1e-6:
        return 0.0
    else:
        return np.dot(v1, v2) / n1 / n2

def word_similarity(model, w1, w2):
    return similarity(model.get_word_vector(w1), model.get_word_vector(w2))

def sentence_similarity(model, t1, t2):
    return similarity(model.get_sentence_vector(t1), model.get_sentence_vector(t2))

In [ ]:
def display_topics_old(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
      print("Topic %d:" % (topic_idx))
      print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
def display_topics(model, feature_names, no_top_words):
    topic_list=[]
    topic_list_2=[]   
    no_top_words=1000  
    for topic_idx, topics in enumerate(model.components_):
      sort_topix=topics.argsort()[:-no_top_words - 1:-1]
      to_idx=[topic_idx]    
      top_list=[]
      print("Topic %d:" % (topic_idx))
      line=[]       
      for i in sort_topix:
        top_list=[]       
        top_list.append(topic_idx)
        top_list.append(feature_names[i])
        top_list.append(topics[i])
        topic_list.append(top_list)
        line.append(feature_names[i])        
     # to_idx.append(topic_list)
      print(line[:10])           
  #  topic_list_2.append(to_idx)                     
#        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    return pd.DataFrame(topic_list,columns=['topic','topic_word','topic_val'])#,topic_list_2

In [ ]:
# облако слов топиков для модели gensim
def plotWordCloud(topic_number,topics,head):
    """
        Строит визуализацию слов на основе текстов топиков из Gensim
    """
    # получаем частоты и слова топика    
    text = dict(topics[topic_number][1])
    #print(head+str(topic_number))    
    # строим облако слов
    wordcloud = WordCloud(background_color="white", max_words=100, width=900, height=900, collocations=False)
    wordcloud = wordcloud.generate_from_frequencies(text)
#    wordcloud.generate(long_string)
    plt.figure(figsize=(15, 10))
    plt.title((head+" номер {}").format(topic_number))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off");

In [ ]:
# облако слов топиков для модели sklearn
def plotWordCloud_2(topic_number,topics):
    """
        Строит визуализацию слов на основе текстов топиков из Sklearn
    """
    # получаем частоты и слова топика
    
#    text = dict(topics[topic_number][1])
    topic=topic_number
    words=topics[topics['topic']==topic]['topic_word'].values.astype('U')
    long_string =','.join(words)
    # строим облако слов
    wordcloud = WordCloud(background_color="white", max_words=100, width=900, height=900, collocations=False)
  #  wordcloud = wordcloud.generate_from_frequencies(long_string)
    wordcloud.generate(long_string)
    plt.figure(figsize=(15, 10))
    plt.title("Топик номер {}".format(topic_number))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off");

In [ ]:
nltk_download("punkt")
!wget https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt
with open ("stop_ru.txt", "r") as stop_ru:
    rus_stops = [word.strip() for word in stop_ru.readlines()]
punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'    
filter_token = rus_stops + list (punctuation)
from pymorphy2 import MorphAnalyzer
parser = MorphAnalyzer()

def preprocess_sklearn(input_text):
    text = input_text.lower()
    tokenized_text = word_tokenize(text)
    clean_text = [word for word in tokenized_text if word not in filter_token]
    lemmatized_text = [parser.parse(word)[0].normal_form for word in clean_text]
    out_text = " ".join(lemmatized_text)    
    return out_text

def preprocess_gemsim(input_text):
    text = input_text.lower()
    tokenized_text = word_tokenize(text)
    clean_text = [word for word in tokenized_text if word not in filter_token]
    lemmatized_text = [parser.parse(word)[0].normal_form for word in clean_text]    
    return lemmatized_text

In [ ]:
def get_topic(words, lda):
    """
    """
    bag = lda.id2word.doc2bow(words)
    topics = lda.get_document_topics(bag)
    topic_dictionary = {}
    for topic in topics:
        topic_dictionary[topic[1]] = str((topic[0])) 
    main_probability = max(topic_dictionary)
    main_topic = topic_dictionary[main_probability]
    return [main_topic, main_probability]

In [ ]:
def lda_model_gensim(corpus,dictionary,num_topics):
  ldamodel = models.ldamodel.LdaModel(
    corpus, 
    id2word=dictionary, 
    eval_every=20, 
    num_topics=num_topics, 
    passes=5
    )
  return ldamodel

In [ ]:
def topic_file(topics_gensim_np,file_name):
  list_topic=[]
  for ind_topiс,topiс in enumerate(topics_gensim_np):
    list_word=topiс[1]
    for words in list_word:
      topic_word=[]
      topic_word.append(ind_topiс)
      topic_word.append(words[0])
      topic_word.append(words[1])
      list_topic.append(topic_word)
  topic_gensim=pd.DataFrame(list_topic,columns=['topic','word_in topic','weght_topic'])
  topic_gensim.to_csv(file_name+'.csv',index=False)
  return topic_gensim

In [ ]:
def topics_max(mod_data,model):
  top_n=[]
  for i in range(mod_data.shape[0]):
#for i in range(100):
    top_max=[]
    topic_list=model.transform(tfidf[i,:])
    top_max.append(np.max(topic_list))
    top_max.append(np.argmax(topic_list))  
    top_n.append(top_max)
  return np.array(top_n)

In [ ]:
#nltk.download("stopwords") # поддерживает удаление стоп-слов
#nltk.download('wordnet') # проводит лемматизацию

In [ ]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [ ]:
def topics_max(mod_data,model):
  top_n=[]
  for i in range(mod_data.shape[0]):
#for i in range(100):
    top_max=[]
    topic_list=model.transform(tfidf[i,:])
    top_max.append(np.max(topic_list))
    top_max.append(np.argmax(topic_list))  
    top_n.append(top_max)
  return np.array(top_n)

In [ ]:
import plotly.express as px

## Импорт данных

In [ ]:
with open("/content/anek_utf8.txt", "r") as file1:
    # итерация по строкам
  n=0
  lines=[]
  for line in file1:
    if line>" ":
      lines.append(line[:-2])
      n=n+1
#  print(n)
  line_new=[]
for n in range(len(lines)-1):
  if lines[n][:15]=='<|startoftext|>':
    if lines[n+1][:15]!='<|startoftext|>':
      line_new.append(lines[n][15:]+' '+lines[n+1][15:])
    else:
      line_new.append(lines[n][15:])  
ds_line=pd.DataFrame(line_new,columns=['text_base'])              

In [ ]:
ds_line.head()

### Обрабока текста

Проведем токенизацию и лемматизацию текста с помощью библиотеки pymorphy2. С помощью нее можно приводить слова в начальную форму. Получим данные для формирования "мешка слов".

In [ ]:
ds_line["text_processed_new"] = ds_line["text_base"].apply(lambda text: preprocess_sklearn(text))
ds_line["text_processed"] = ds_line["text_base"].apply(lambda text: preprocess_gensim(text))

In [ ]:
ds_line.head()

Сохраним данные для дальнейшей работы

In [ ]:
ds_line['text_base'].to_csv('anekdot_texts.csv',index=False, index_label=False)
ds_line.to_csv('anekdot_pr.csv',index=False, index_label=False)

## Использование предобработанных данных для мешка слов (BOW)

In [ ]:
import gensim
from gensim import corpora, models
from ast import literal_eval
ds_line = pd.read_csv('anekdot_pr.csv',converters={"text_processed": literal_eval})

### Словарь непосредствено из (bag of words) BOW

In [ ]:
dictionary = gensim.corpora.Dictionary(ds_line["text_processed"].values) # составляем словарь из терминов             
print('Размер словаря до фильтрации: {}'.format(len(dictionary)))
#gensim_dictionary.filter_extremes(no_above=0.1, no_below=20)
dictionary.filter_extremes(no_below=3, no_above=0.4, keep_n=3*10**6) # практически не фильтрует
#dictionary.filter_extremes(no_above=0.1, no_below=20)

dictionary.compactify()
print('Размер словаря после фильтрации: {}'.format(len(dictionary)))
print(dictionary)

Размер словаря до фильтрации: 70843
Размер словаря после фильтрации: 25807
Dictionary(25807 unique tokens: ['актуальный', 'бог', 'вопрос', 'дефицит', 'дзюба']...)


### Создаем корпус

In [ ]:
corpus = [dictionary.doc2bow(text) for text in ds_line['text_processed']]

## Получение словаря и корпуса на основании биграмм

In [ ]:
# Build the bigram  models
texts=(ds_line["text_processed"])
bigram = models.Phrases(texts, min_count=3, threshold=5)
bigram_mod = models.phrases.Phraser(bigram)
texts_bi = make_bigrams(texts)
dictionary_bi = corpora.Dictionary(texts_bi)                 # составляем словарь из терминов с учетом биграмм слов предлжений
print('Размер словаря до фильтрации: {}'.format(len(dictionary_bi)))
dictionary_bi.filter_extremes(no_below=3, no_above=0.4, keep_n=3*10**6)
print('Размер словаря после фильтрации: {}'.format(len(dictionary_bi)))
corpus_bi = [dictionary_bi.doc2bow(text) for text in texts_bi]  # составляем корпус документов по словарю полученному с использованием биграмм

Размер словаря до фильтрации: 83999
Размер словаря после фильтрации: 37984


### Пример токенов из полученых словарей

In [ ]:
print(dictionary_bi)
print(dictionary)

## Формирование тем, на основании модели Latent Dirichlet allocation (LDA) из Gensim

количество топиков пока произвольно 20, вопрос числа топиков открытый. 

In [ ]:
number_topics=20 

In [ ]:
ldamodel_base=lda_model_gensim(corpus,dictionary,number_topics)
ldamodel_base.save('lda_20_base')
ldamodel_bi=lda_model_gensim(corpus_bi,dictionary_bi,number_topics)
ldamodel_bi.save('lda_20_bi')

### Получение сохранение топиков, для дальнейшей визуализации

In [ ]:
topics_gensim_base = ldamodel_base.show_topics(num_topics=number_topics, num_words=100,formatted=False)
topics_gensim_bi = ldamodel_bi.show_topics(num_topics=number_topics, num_words=100,formatted=False)
with open('topics_base_cloud', 'wb') as f:
    pickle.dump(topics_gensim_base, f)
with open('topics_bi_cloud', 'wb') as f:
    pickle.dump(topics_gensim_bi, f)

topics_base=pd.DataFrame(topics_gensim_base)
topics_base.columns=['topic num','topic_word']
topics_bi=pd.DataFrame(topics_gensim_bi)
topics_bi.columns=['topic num','topic_word']

In [ ]:
#topics_base.to_csv('topic_gensim_base.csv',index=False)
#topics_bi.to_csv('topic_gensim_bi.csv',index=False)

## Визуализация

В библиотеке gensim есть встроенная интерактивная визаулизация расстояния между темами.
Применим ее для обеих моделей

In [ ]:
vis_base = gensimvis.prepare(ldamodel_base, corpus, dictionary)

In [ ]:
vis_bi = gensimvis.prepare(ldamodel_bi, corpus_bi, dictionary_bi)

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
vis_base

In [ ]:
vis_bi

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7      0.196563  0.066397       1        1  8.173471
5      0.177412 -0.063959       2        1  6.367893
10    -0.170029  0.329922       3        1  6.272427
16     0.123890  0.021254       4        1  6.034704
11     0.144613  0.023943       5        1  5.152347
19     0.134491  0.056501       6        1  4.974916
6      0.059737 -0.021964       7        1  4.766146
14     0.053475  0.028162       8        1  4.662136
18     0.024300  0.010867       9        1  4.607687
8      0.040930 -0.013847      10        1  4.564252
1      0.061022 -0.152231      11        1  4.557942
13    -0.014604 -0.068855      12        1  4.550547
4     -0.133300 -0.152846      13        1  4.513166
0     -0.032999  0.135781      14        1  4.492763
17    -0.158618 -0.078000      15        1  4.456513
3     -0.104840  0.027386      16        1  4.446377
15    -0.089683 -0.101210      17        1  4.441501
12    -0.113249 -0.000267      18        1  4.394881
9     -0.183099 -0.082801      19        1  4.315318
2     -0.016009  0.035767      20        1  4.255014, topic_info=           Term          Freq         Total Category  logprob  loglift
23           ''  14209.000000  14209.000000  Default  30.0000  30.0000
24           ``  11179.000000  11179.000000  Default  29.0000  29.0000
321        жена   6371.000000   6371.000000  Default  28.0000  28.0000
27      женщина   4895.000000   4895.000000  Default  27.0000  27.0000
193     девушка   3919.000000   3919.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1776  согласный    191.833943    192.717569  Topic20  -5.4539   3.1525
243          ..    771.766068  15067.285762  Topic20  -4.0619   0.1855
74          ...    354.912702  11924.610024  Topic20  -4.8387  -0.3574
48         один    243.178435   3001.333035  Topic20  -5.2167   0.6441
125       место    199.340633    811.857468  Topic20  -5.4155   1.7528

[860 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
23        3  0.999906        ''
2443     20  0.997358     ''_.-
243       1  0.126632        ..
243       2  0.110969        ..
243       3  0.007765        ..
...     ...       ...       ...
1080      7  0.996235   экзамен
1843      7  0.994194        эх
3910      9  0.994109  являться
1056      9  0.996710      язык
1641      8  0.997257      яйцо

[1320 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 6, 11, 17, 12, 20, 7, 15, 19, 9, 2, 14, 5, 1, 18, 4, 16, 13, 10, 3])

### преобразование сохраненных топиков в более удобный формат хранения и использования в дальнейшем

In [ ]:
topics_gensim_base_np=topics_base.values
topics_gensim_bi_np=topics_bi.values
topic_gensim_base=topic_file(topics_gensim_base_np,'topic_base')
topic_gensim_bi=topic_file(topics_gensim_bi_np,'topic_bi')

In [ ]:
topic_gensim_bi.head()

,topic,word_in topic,weght_topic
0,0,мама,0.040519
1,0,папа,0.031565
2,0,сын,0.018772
3,0,отец,0.016297
4,0,бояться,0.015293


### Посмотрим примеры визуализации полученных топиков

In [ ]:
with open('/content/topics_base_cloud', 'rb') as f:
    lda_topics_base = pickle.load(f)
with open('/content/topics_bi_cloud', 'rb') as f:
    lda_topics_bi = pickle.load(f)
for i in range(20):
  #print(i)
  plotWordCloud(topic_number=i, topics=lda_topics_base,head='Топики base ')
  plotWordCloud(topic_number=i, topics=lda_topics_bi,head='Топики bi')    

## Проверим такую идею:

Назначим каждому документу наиболее подходящий (наиболее вероятный) для него топик, для каждой из моделей(base и bi).

In [ ]:
ds_line["lda_base"] = ds_line["text_processed"].apply(get_topic, lda=ldamodel_base)
ds_line["lda_bi"] = ds_line["text_processed"].apply(get_topic, lda=ldamodel_bi)
ds_line["topic_base"] = ds_line["lda_base"].str[0]
ds_line["probability_base"] = ds_line["lda_base"].str[1]
del ds_line["lda_base"]
ds_line["topic_bi"] = ds_line["lda_bi"].str[0]
ds_line["probability_bi"] = ds_line["lda_bi"].str[1]
del ds_line["lda_bi"]
del ds_line["text_processed_new"]
del ds_line["text_processed"]

In [ ]:
ds_line[['topic_base','topic_bi']].head(10)

Сохраним полученные присвоения для дальнейшего использования

In [ ]:
ds_line.to_csv('anekdot_topics.csv',index=False, index_label=False)

## Небольшой разведочный анализ полученного результата

In [ ]:
ds_line=pd.read_csv('/content/anekdot_topics.csv')
ds_line.groupby(['topic_base'])['text_base'].count()
ds_line.groupby(['topic_bi'])['text_base'].count()
ds_line.groupby(['topic_nmf'])['text_base'].count()
ds_line.groupby(['topic_lda'])['text_base'].count()

In [ ]:
ds_line.tail()

,text_base,topic_base,probability_base,topic_bi,probability_bi,topic_nmf,probability_nmf,topic_lda,probability_lda
118169,"- Кстати, если на Новый год напишешь письмо не...",7.0,0.324962,14.0,0.287116,14.0,0.050001,4.0,0.395096
118170,"У рукколы есть нормальное русское название - ""...",5.0,0.256250,10.0,0.381250,14.0,0.019956,17.0,0.419386
118171,Хищения в особо крупных размерах продолжаются ...,17.0,0.203333,12.0,0.183623,19.0,0.003331,7.0,0.215942
118172,"Конечно, на вечере встречи выпускников иногда ...",18.0,0.307837,1.0,0.221138,19.0,0.005323,0.0,0.180571
118173,- Женской дружбы нет. Это миф.- Дружбы между,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ds_line.groupby(['topic_base'])['text_base'].count()

In [ ]:
ds_line.groupby(['topic_bi'])['text_base'].count()

Оценим степень совпадения топиков полученных по разным исходным данным в библиотеке Gensim.

Определим "устойчивые" - наиболее популярные пары топиков, которые назначаются одному документу.

Поскольку, как мы видим, значение для *probability* в некоторых случаях не высокая, ограничимся только теми записями у которых оно выше **0.5**

In [ ]:
ds_line_lite=ds_line[(ds_line['probability_base']>0.5)&(ds_line['probability_bi']>0.5)]
ds_top=ds_line_lite.groupby(['topic_base','topic_bi'])['text_base'].count().reset_index()
ds_top.columns=['topic_base',	'topic_bi',	'rec_counts']

In [ ]:
ds_top=ds_line_lite.groupby(['topic_base','topic_bi'])['text_base'].count().reset_index()
ds_top.columns=['topic_base',	'topic_bi',	'rec_counts']
#ds_top.head()

In [ ]:
ds_top['rec_counts'].describe()

count    394.000000
mean       9.527919
std       15.307052
min        1.000000
25%        4.000000
50%        6.000000
75%       10.000000
max      181.000000
Name: rec_counts, dtype: float64

Видно что разница между минимальным (1) и максимальным значением (181), для более наглядной визуализации полученного результата  воспользуемся функцией десятичного логарифмом от *rec_counts* 



In [ ]:
ds_top['log']=np.log10(ds_top['rec_counts'])
topics_base_bi = pd.pivot_table(ds_top,
               index='topic_base',
               values='log',
               columns='topic_bi',
               fill_value=0)
topics_base_bi_np=topics_base_bi.values
#topics_base_bi.head()

In [ ]:
fig = px.imshow(topics_base_bi,width=600, height=600, labels = dict(y = "Топики Base", x = "Топики Bi", color = "Legend"))
#fig.update_xaxes(side = "top")
fig.show()

Топики полученных для разных моделей, но имеющие высокую частотность

In [ ]:
plotWordCloud(topic_number=7, topics=lda_topics_base,head='Топики base ')
plotWordCloud(topic_number=7, topics=lda_topics_bi,head='Топики bi ')

Как видим они визуально походи и выделяют сходные токены.

Пример сравнения топиков полученных для разных моделей, но имеющие меньшую частоту

In [ ]:
plotWordCloud(topic_number=5, topics=lda_topics_base,head='Топики base ')
plotWordCloud(topic_number=10, topics=lda_topics_bi,head='Топики bi ')

Видны сходные топики, но их популярность представлена по разному

Топики с низкой частотой

In [ ]:
plotWordCloud(topic_number=9, topics=lda_topics_base,head='Топики base ')
plotWordCloud(topic_number=9, topics=lda_topics_bi,head='Топики bi ')

Видно что между ними практически отсутсвуют общие токены

Другое представление связи между топиками, позволяющее видеть значений логарифма частоты совместной встречаемости

In [ ]:
plt.rcParams['figure.figsize']=15,15
sns.heatmap(topics_base_bi, annot=True, fmt=".1f") 

Следует отметить, что полученные выше результат относится лишь к 3% от общего числа документов (9*394=3546).

Снижение порога до уровня 0.4 или 0.3 - увеличило бы число учитываемых документов, но снижало бы достоверность отнесения документов к одному топику.

# В библиотеке **sklearn** для векторизации на базе весов TF-IDF для токенов в предложениях текста, воспользуемся функцией Tfidfvectorizer.

Модель Tfidfvectorizerсоздает представление исходного массива текстов разреженой матрицей размерности (N - количество тектов, M - мощность словаря), при этом словарь фильтруется отсекая токены имеющие малые значения величины (веса) TF-IDF.

Импротируем созраненные данные, при этом пропустим этап токенизации и лемматизации воспользовавшись текстами подготовленными при работе с Gensim

In [ ]:
ds_line = pd.read_csv('anekdot_pr.csv') #,converters={"text_processed": literal_eval})
from sklearn.feature_extraction.text import TfidfVectorizer
#no_features = 300

#tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=filter)
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,  stop_words=filter)
tfidf = tfidf_vectorizer.fit_transform(ds_line["text_processed_new"].values.astype('U'))

tokens_sklearn = tfidf_vectorizer.get_feature_names_out()
print('Длина словаря',len(tokens_sklearn))

Модель TfidfVectorizer позволяет создавать словари с заранее ограниченной длинной и без такого ограничения. Для нашего исследования ограничение снято.

Полученная мощность словаря сопоставима с той, что была полученно для мешка слов в Gensim

In [ ]:
tokens_sklearn

## Модели получения тем

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
no_topics = 20
no_top_words=10

### тренировка моделей получения числа топиков, для двух подходов выделения топиков.

In [ ]:
# Model lda_tfidf
lda_tfidf = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=1024).fit(tfidf)
# Model NMF
nmf = NMF(n_components=no_topics, random_state=1024, l1_ratio=.5, init='nndsvd').fit(tfidf)

/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning:

Maximum number of iterations 200 reached. Increase it to improve convergence.



### получим распределение топиков для текстов и произведем присвоение полученных значений текстам наиболее популярных топиков, аналогично тому как это было сделано для моделей в Gensim

In [ ]:
top_lda=topics_max(tfidf,lda_tfidf)
top_nmf=topics_max(tfidf,nmf)
ds_topic=pd.read_csv('/content/anekdot_topics.csv')
ds_topic['topic_nmf']=top_nmf[:,1]
ds_topic['probability_nmf']=top_nmf[:,0]
ds_topic['topic_lda']=top_lda[:,1]
ds_topic['probability_lda']=top_lda[:,0]

In [ ]:
top_nmf=topics_max(tfidf,nmf)

In [ ]:
ds_topic=pd.read_csv('/content/anekdot_topics.csv')

In [ ]:
ds_topic['topic_nmf']=top_nmf[:,1]
ds_topic['probability_nmf']=top_nmf[:,0]

ds_topic['topic_lda']=top_lda[:,1]
ds_topic['probability_lda']=top_lda[:,0]

In [ ]:
ds_topic.head(50)

Сохраним для дальнейшего анализа файл содержащий ссылки на топики к которым отнесены анекдоты разными моделями.

In [ ]:
ds_topic.to_csv('anekdot_topics.csv',index=False, index_label=False)

### Визуализация топиков полученный по моделям nmf и lda_tf-idf

In [ ]:
no_top_words=10
topic_lda=display_topics(lda_tfidf, tokens_sklearn, no_top_words)

In [ ]:
topic_nmf=display_topics(nmf, tokens_sklearn, no_top_words)

In [ ]:
topic_nmf.to_csv('topic_nmf.csv',index=False, index_label=False)
topic_lda.to_csv('topic_lda.csv',index=False, index_label=False)

Облака для топиков моделей sklearn

In [ ]:
for i in range(20):
    plotWordCloud_2(topic_number=i, topics=topic_lda)    

In [ ]:
for i in range(20):
    plotWordCloud_2(topic_number=i, topics=topic_nmf)    

## По аналогии с предыдущим случаем, интересно сравнить топики формируемые разными подходами и особенностями библиотек gensim и sklearn.

Импортируем сохраненные ранее сохраненые топики

In [ ]:
topic_base=pd.read_csv('/content/topic_base.csv')
topic_bi=pd.read_csv('/content/topic_bi.csv')
topic_nmf=pd.read_csv('/content/topic_nmf.csv')
topic_lda=pd.read_csv('/content/topic_lda.csv')
#topic_gemsim_np=topic_gemsim.values
#topic_nmf_np=topic_nmf.values


### для получения эмбеддингов используем модель "маленький BERT" описанную в  https://habr.com/ru/post/562064/ .

Из нее получим модели токенизации получения эбеддингов

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()  # uncomment it if you have a GPU

## Для примера сравним топики из моделей topic_base (Gensim) и topic_nmf (Sklearn). 

И ограничимся 15 первыми словами каждого из топиков (иначе ресурсов бесплатного colab может не хватить). 

Сравнение векторов полученных из эмбеддингов, осуществим на основании косинусного расстояния.

In [ ]:
plt.rcParams['figure.figsize']=15,15
topic_sim=[]
for i in range(20):
  v1 = ' '.join(topic_base[topic_base['topic']==i]['topic_word'].values[:15])
  v1_emb=embed_bert_cls(v1, model, tokenizer)
#  print(v1)
  for m in range(20):
    sim_work=[]
    v2 = ' '.join(topic_nmf[topic_nmf['topic']==m]['topic_word'].values[:15])
    v2_emb=embed_bert_cls(v2, model, tokenizer)
    sim=similarity(v1_emb, v2_emb)
    sim_work.append(i)
    sim_work.append(m)
    sim_work.append(sim)
    topic_sim.append(sim_work)
#topic_sim
topic_sim_ds=pd.DataFrame(topic_sim,columns=['base','nmf','cos_sim'])
glue = topic_sim_ds.pivot('base', 'nmf', 'cos_sim')
sns.heatmap(glue, annot=True, fmt=".1f")        

In [ ]:
fig = px.imshow(glue,width=600, height=600)
fig.show()

In [ ]:
ds_line=pd.read_csv('/content/anekdot_topics.csv')
topic=pd.read_csv('/content/topic_nmf.csv')

In [ ]:
ds_line.head()

In [ ]:
topic_nmf=pd.read_csv('/content/topic_nmf.csv')
topic_lda=pd.read_csv('/content/topic_lda.csv')
topic_base=pd.read_csv('/content/topic_base.csv')
topic_bi=pd.read_csv('/content/topic_bi.csv')

In [ ]:
topic_bi.head()

In [ ]:
topic[topic['topic']==].head()

In [ ]:
topic_nnmf_0=ds_line[ds_line['topic_nmf']==0]['text_base']

In [ ]:
joke_generator(ds_line,9,10,'topic_nmf')

In [ ]:
def joke_generator(data,topic_id,num,topic_model='topic_lda',):
  if topic_model=='topic_base': 
    data_topic=topic_base
  if topic_model=='topic_bi': 
    data_topic=topic_bi
  if topic_model=='topic_nmf': 
    data_topic=topic_nmf
  if topic_model=='topic_lda': 
    data_topic=topic_lda
  word_in_topic=data_topic[data_topic['topic']==topic_id]['topic_word'].values[:10]
  print('токены топика:',", ".join(word_in_topic) )
  print()

#  plotWordCloud_2(topic_number=topic_id, topics=data_topic)
  jokes=data[data[topic_model]==topic_id]['text_base'].values  
  for i, joke in enumerate(jokes[:num]):
    print('анекдот',i,joke)

In [ ]:
print(ds_line.groupby(['topic_base'])['text_base'].count())
print(ds_line.groupby(['topic_bi'])['text_base'].count())
print(ds_line.groupby(['topic_nmf'])['text_base'].count())
print(ds_line.groupby(['topic_lda'])['text_base'].count())

Покажим в сравнении топики для высокого значения косинусного расстояния

In [ ]:
plotWordCloud_2(topic_number=0, topics=topic_nmf)
plotWordCloud(topic_number=16, topics=lda_topics_base,head='Топики base ')

In [ ]:
plotWordCloud_2(topic_number=2, topics=topic_nmf)
plotWordCloud(topic_number=3, topics=lda_topics_base,head='Топики base ')

In [ ]:
plotWordCloud_2(topic_number=6, topics=topic_nmf)
plotWordCloud(topic_number=7, topics=lda_topics_base,head='Топики base ')